# SQLite
## Maximum 50 points out of 100 (total exam)

## You are expected to use a single query for each of the exercise below:


1. Find all records where the total_bill exceeds $50. (2 points)
2. Calculate the average tip given. (2 points)
3. Find the maximum total_bill paid. (2 points)
4. Count how many people were served on each day. (4 points)
5. Calculate the average total_bill grouped by size. (4 points)
6. Find the total tip amount for smokers vs. non-smokers. (4 points)
7. Determine the average tip percentage (tip/total_bill) for each day. (4 points)
8. List all total_bill values with the corresponding day and time. (4 points)
9. Find the total number of observations for each gender. (4 points)
10. Retrieve all tips given during Dinner on Saturday. (4 points)
11. Find the largest total_bill for each gender. (4 points)
12. Find the average tip for parties of size greater than 4. (4 points)
13. Find all records from where the day is either Friday or Saturday, and the time is Lunch or Dinner. (4 points)
14. List all records where the tip percentage is higher than 20%. (4 points)

## Bonus question (10 points)


15. Use a subquery (query inside query) to find the record with the second highest tip. (you need to use a single query)

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
conn = sqlite3.connect("tips.db")
cur= conn.cursor()

In [ ]:
query = "SELECT name FROM sqlite_master WHERE type='table'"
cur.execute(query)
cur.fetchall()

[('Observation',), ('Sex',), ('Smoker',), ('Day',), ('Time',)]

In [ ]:
query = "PRAGMA table_info(Observation)"
cur.execute(query)
cur.fetchall()

[(0, 'total_bill', 'FLOAT', 0, None, 0),
 (1, 'tip', 'FLOAT', 0, None, 0),
 (2, 'size', 'BIGINT', 0, None, 0),
 (3, 'sex_id', 'BIGINT', 0, None, 0),
 (4, 'smoker_id', 'BIGINT', 0, None, 0),
 (5, 'day_id', 'BIGINT', 0, None, 0),
 (6, 'time_id', 'BIGINT', 0, None, 0)]

In [ ]:
query = '''
SELECT * FROM Observation
'''
pd.read_sql(query, conn)

,total_bill,tip,size,sex_id,smoker_id,day_id,time_id
0,16.99,1.01,2,1,1,3,1
1,10.34,1.66,3,0,1,3,1
2,21.01,3.50,3,0,1,3,1
3,23.68,3.31,2,0,1,3,1
4,24.59,3.61,4,1,1,3,1
...,...,...,...,...,...,...,...
239,29.03,5.92,3,0,1,2,1
240,27.18,2.00,2,1,0,2,1
241,22.67,2.00,2,0,0,2,1
242,17.82,1.75,2,0,1,2,1


In [ ]:
query = '''
SELECT * FROM Observation
WHERE Observation.total_bill > 50
'''
pd.read_sql(query, conn)

,total_bill,tip,size,sex_id,smoker_id,day_id,time_id
0,50.81,10.0,3,0,0,2,1


In [ ]:
query = '''
SELECT Observation.tip, AVG(Observation.tip) AS average_tip
FROM Observation
'''
pd.read_sql(query, conn)

,tip,average_tip
0,1.01,2.998279


In [ ]:
query = '''
SELECT MAX(Observation.total_bill) AS maximum_total_bill
FROM Observation
'''
pd.read_sql(query, conn)

,maximum_total_bill
0,50.81


In [ ]:
query = '''
SELECT
Day.day, SUM(Observation.size) AS people_served
FROM Observation
JOIN Day ON Day.day_id=Observation.day_id
GROUP BY Day.day
'''
pd.read_sql(query, conn)

,day,people_served
0,Fri,40
1,Sat,219
2,Sun,216
3,Thur,152


In [ ]:
query = '''
SELECT ROUND(AVG(Observation.total_bill),2) AS average_total_bill,Observation.size
FROM Observation
GROUP BY Observation.size
'''
pd.read_sql(query, conn)

,average_total_bill,size
0,7.24,1
1,16.45,2
2,23.28,3
3,28.61,4
4,30.07,5
5,34.83,6


In [ ]:
query = '''
SELECT * FROM Smoker
'''
pd.read_sql(query, conn)

,smoker_id,smoker
0,0,Yes
1,1,No


In [ ]:

query = """
SELECT Smoker.smoker,
       SUM(Observation.tip) as total_tip
FROM Observation
JOIN Smoker ON Observation.smoker_id = Smoker.smoker_id
GROUP BY Smoker.smoker_id
"""
pd.read_sql(query, conn)

,smoker,total_tip
0,Yes,279.81
1,No,451.77


In [ ]:
query = """
SELECT Day.day,
       ROUND((AVG((Observation.tip)*100/(Observation.total_bill))), 2) AS AVG_tip_percentage
FROM Observation
JOIN Day ON Observation.day_id = Day.day_id
GROUP BY Day.day
"""
pd.read_sql(query, conn)

,day,AVG_tip_percentage
0,Fri,16.99
1,Sat,15.32
2,Sun,16.69
3,Thur,16.13


In [ ]:
query = """
SELECT Observation.total_bill, Day.day, Time.time
FROM Observation
JOIN Day ON Observation.day_id = Day.day_id
JOIN Time ON Observation.time_id = Time.time_id
"""
pd.read_sql(query, conn)

,total_bill,day,time
0,16.99,Sun,Dinner
1,10.34,Sun,Dinner
2,21.01,Sun,Dinner
3,23.68,Sun,Dinner
4,24.59,Sun,Dinner
...,...,...,...
239,29.03,Sat,Dinner
240,27.18,Sat,Dinner
241,22.67,Sat,Dinner
242,17.82,Sat,Dinner


In [ ]:
query = '''
SELECT * FROM Sex
'''
pd.read_sql(query, conn)

,sex_id,sex
0,0,Male
1,1,Female


In [ ]:
query = """
SELECT Sex.sex, COUNT(*) AS obs_num
FROM Observation
JOIN Sex ON Observation.sex_id = Sex.sex_id
GROUP BY Sex.sex
"""
pd.read_sql(query, conn)

,sex,obs_num
0,Female,87
1,Male,157


In [ ]:
query = """
SELECT SUM(Observation.tip) AS total_tip
FROM Observation
JOIN Day ON Observation.day_id = Day.day_id
JOIN Time ON Observation.time_id = Time.time_id
WHERE Day.day = 'Sat' AND Time.time = 'Dinner'
GROUP BY Day.day, Time.time
"""
pd.read_sql(query, conn)

,total_tip
0,260.4


# **<font color='red'> The task required to retrieve all tips given during Dinner on Saturday. There is no need to culculate the sum of tip and group it by day and time. Minus 3 points </font>**


In [ ]:
query = """
SELECT Sex.sex, MAX(Observation.total_bill) AS max_total_bill
FROM Observation
JOIN Sex ON Observation.sex_id = Sex.sex_id
GROUP BY Sex.sex
"""
pd.read_sql(query, conn)

,sex,max_total_bill
0,Female,44.30
1,Male,50.81


In [ ]:
query = """
SELECT Observation.size, AVG(Observation.tip) AS AVG_tip
FROM Observation
WHERE Observation.size > 4
GROUP BY Observation.size
"""
pd.read_sql(query, conn)


,size,AVG_tip
0,5,4.028
1,6,5.225


# **<font color='red'> You need just find the average tip for the size greater than 4. There is no need to group by size. Minus 1 point </font>**

In [ ]:
query = """
SELECT * FROM Observation
JOIN Day ON Observation.day_id = Day.day_id
JOIN Time ON Observation.time_id = Time.time_id
WHERE (Day.day = 'Fri' OR Day.day = 'Sat')
AND (Time.time = 'Lunch' OR Time.time = 'Dinner')
"""
pd.read_sql(query, conn)

,total_bill,tip,size,sex_id,smoker_id,day_id,time_id,day_id,day,time_id,time
0,5.75,1.00,2,1,0,1,1,1,Fri,1,Dinner
1,8.58,1.92,1,0,0,1,0,1,Fri,0,Lunch
2,10.09,2.00,2,1,0,1,0,1,Fri,0,Lunch
3,11.35,2.50,2,1,0,1,1,1,Fri,1,Dinner
4,12.03,1.50,2,0,0,1,1,1,Fri,1,Dinner
...,...,...,...,...,...,...,...,...,...,...,...
101,39.42,7.58,4,0,1,2,1,2,Sat,1,Dinner
102,44.30,2.50,3,1,0,2,1,2,Sat,1,Dinner
103,48.27,6.73,4,0,1,2,1,2,Sat,1,Dinner
104,48.33,9.00,4,0,1,2,1,2,Sat,1,Dinner


In [ ]:
query = """
SELECT ROUND((Observation.tip * 100 / Observation.total_bill),2) AS tip_percentage, * FROM Observation
WHERE tip_percentage > 20
"""
pd.read_sql(query, conn)

,tip_percentage,total_bill,tip,size,sex_id,smoker_id,day_id,time_id
0,22.81,8.77,2.00,2,0,1,3,1
1,21.85,14.78,3.23,2,0,1,3,1
2,20.36,14.83,3.02,2,1,1,3,1
3,22.77,16.29,3.71,3,0,1,3,1
4,20.62,16.97,3.50,3,1,1,3,1
5,22.77,17.92,4.08,2,0,1,2,1
6,21.95,13.94,3.06,2,0,1,3,1
7,22.49,22.23,5.00,2,0,1,3,1
8,25.27,10.29,2.60,2,1,1,3,1
9,20.56,18.29,3.76,4,0,0,2,1


In [ ]:
#Bonus

query = """
SELECT tip FROM Observation
WHERE tip < (SELECT MAX(tip) FROM Observation)
ORDER BY tip DESC
LIMIT 1
"""
pd.read_sql(query, conn)

,tip
0,9.0


# **<font color='red'> Incorrect solution. </font>**